# Expression remapper

This notebook is a simple app that allows users to map different facial expressions and motions to keyboard inputs. I aim to use this to play a video game like Dark Souls as a test.

Using MediaPipe, and following their [face landmark detection guide](https://developers.google.com/mediapipe/solutions/vision/face_landmarker/python).

## Install dependencies

#### Mediapipe
Using for computer vision.

In [ ]:
!pip install mediapipe
# opencv as well

#### PyAutoGUI
Allows us to simulate keyboard events through code.

In [23]:
!pip install pyautogui

### Virtual Gamepad
Allows us to emulate virtual game controller inputs.

NOTE: it will install a driver that is needed. It also is only known to be stable on Windows, and it says 'experimental' for Linux.

In [ ]:
!pip install vgamepad

### Pynput
Allows us to emulate keyboard inputs

In [1]:
!pip install pynput

     ---------------------------------------- 0.0/89.2 kB ? eta -:--:--
     ---------------------------------------- 89.2/89.2 kB 5.3 MB/s eta 0:00:00


## Import dependencies

In [2]:
# Mediapipe for face landmark detection
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision

# OpenCV for drawing utilities and webcam input
import cv2

# Generally useful for math
import math

# PyAutoGUI for simulating keyboard inputs
from pyautogui import press, typewrite, hotkey # only really need 'press'

# Pynput for simulating keyboard inputs
from pynput.keyboard import Key, Controller

## Load the model

In [3]:
model_path = 'models/face_landmarker.task'

## Utility functions for extracting head orientation

In [28]:
#!! import math

# Toggles whether the remapping is currently active. If paused, no keypresses will be simulated.
remapping_paused = True

# State variables
keyboard = Controller()
moving_up = False
moving_down = False
moving_left = False
moving_right = False

# Real yaw pitch and roll
yaw = 0.0 # Rotate left/right
pitch = 0.0 # Tilt up/down
roll = 0.0 # Tilt left/right

# Calibrated yaw pitch and roll values
calib_yaw = 0
calib_pitch = 0
calib_roll = 0

# The rest positions, used to calculate the calibrated values
rest_yaw = 0.0
rest_pitch = 0.0
rest_roll = 0.0

# Extract the yaw, pitch, and roll of the head from the facial transformation matrix
# (yaw, pitch, and roll are easier for me to work with than a matrix)
def update_yaw_pitch_roll(mtrix):
    # Get access to the global variables
    global yaw
    global pitch
    global roll

    # Calculate the yaw pitch and roll from the transformation matrix (conversion)
    if mtrix[0][0] == 1.0:
        yaw = math.atan2(mtrix[0][2], mtrix[2][3])
        pitch = 0
        roll = 0

    elif mtrix[0][0] == -1.0:
        yaw = math.atan2(mtrix[0][0], mtrix[2][3])
        pitch = 0
        roll = 0

    else:
        yaw = math.atan2(-mtrix[2][0], mtrix[0][0])
        pitch = math.atan2(-mtrix[1][2], mtrix[1][1])
        roll = math.asin(mtrix[1][0])

    # Calculate the CALIBRATED yaw pitch roll values
    global calib_yaw
    global calib_pitch
    global calib_roll
    calib_yaw = yaw - rest_yaw
    calib_pitch = pitch - rest_pitch
    calib_roll = roll - rest_roll
    
#     # Yaw (look left/right)
#     print("yaw is: {}".format(math.degrees(calib_yaw)))
#     if calib_yaw > 0:
#         print("looking left")
#     elif calib_yaw < 0:
#         print("looking right")

#         # Pitch (look up/down)
#         print("pitch is: {}".format(math.degrees(calib_pitch)))
#         if calib_pitch > 0:
#             print("looking down")
#         elif calib_pitch < 0:
#             print("looking up")

#         # Roll (rilt left/right)
#         print("roll is: {}".format(math.degrees(calib_roll)))
#         if calib_roll > 0:
#             print("looking rolled right")
#         elif calib_roll < 0:
#             print("looking rolled left")

    if not remapping_paused:
        global moving_left
        global moving_right
        global moving_up
        global moving_down
        
        #print(math.degrees(calib_pitch))
        thresh = 10.0
#         # Look Left / Right
#         if math.degrees(calib_yaw) > thresh: # Head is facing left
#             keyboard.release('d')
#             moving_right = False
#             if not moving_left:
#                 keyboard.press('a')
#                 moving_left = True
#         elif math.degrees(calib_yaw) < -thresh: # Head is facing right
#             keyboard.release('a')
#             moving_left = False
#             if not moving_right:
#                 keyboard.press('d')
#                 moving_right = True
#         else: # Head is in neutral position
#             keyboard.release('a')
#             moving_left = False
#             keyboard.release('d')
#             moving_right = False
        
        
        # Tilt Left / Right
        if math.degrees(calib_roll) > thresh: # Head is tilted right
            keyboard.release('a')
            moving_left = False
            if not moving_right:
                keyboard.press('d')
                moving_right = True
        elif math.degrees(calib_roll) < -thresh: # Head is tilted left
            keyboard.release('d')
            moving_right = False
            if not moving_left:
                keyboard.press('a')
                moving_left = True
        else: # Head is in neutral position
            keyboard.release('a')
            moving_left = False
            keyboard.release('d')
            moving_right = False
        
        
        # Up / Down
        if math.degrees(calib_pitch) > thresh:
            keyboard.release('s')
            moving_up = False
            if not moving_down:
                keyboard.press('w')
                moving_down = True
        elif math.degrees(calib_pitch) < -thresh:
            keyboard.release('w')
            moving_down = False
            if not moving_up:
                keyboard.press('s')
                moving_up = True
        else: # Head is in neutral position
            keyboard.release('w')
            moving_down = False
            keyboard.release('s')
            moving_up = False
    
def calibrate_yaw_pitch_roll():
    # Get access to global rest pos variables (I don't like python..)
    # then set them to the current yaw, pitch, and roll of the user's head.
    global yaw
    global pitch
    global roll
    global rest_yaw
    global rest_pitch
    global rest_roll
    rest_yaw = yaw
    rest_pitch = pitch
    rest_roll = roll
    
    # Print results
    print("resting yaw is: " + str(rest_yaw))
    print("resting pitch is: " + str(rest_pitch))
    print("resting roll is: " + str(rest_roll))
    

## Main program

In [29]:
#!! import mediapipe as mp

BaseOptions = mp.tasks.BaseOptions
FaceLandmarker = mp.tasks.vision.FaceLandmarker
FaceLandmarkerOptions = mp.tasks.vision.FaceLandmarkerOptions
FaceLandmarkerResult = mp.tasks.vision.FaceLandmarkerResult
VisionRunningMode = mp.tasks.vision.RunningMode
    

# Create a face landmarker instance with the live stream mode:
def print_result(result: FaceLandmarkerResult, output_image: mp.Image, timestamp_ms: int):
    
    # If any landmarks have been detected..
    if result.face_landmarks:
        
        # Get the matrix from face landmarker
        transf_matrix = result.facial_transformation_matrixes[0]
        update_yaw_pitch_roll(transf_matrix)   
    

options = FaceLandmarkerOptions(
    base_options=BaseOptions(model_asset_path=model_path),
    running_mode=VisionRunningMode.LIVE_STREAM,
    result_callback=print_result,
    output_facial_transformation_matrixes=True, ## TRY CHANGING THIS
    output_face_blendshapes=False) ## TRY CHANGING THIS

with FaceLandmarker.create_from_options(options) as landmarker:
    # The landmarker is initialized. Use it here.
    # Use OpenCV’s VideoCapture to start capturing from the webcam.
    video = cv2.VideoCapture(0)
    
    # Initialise the frame timestamp (MAKE THIS IN ms?)
    frame_timestamp = 0
    
    # Tell the user to calibrate
    print("---- Calibration -----")
    print("Please look straight ahead at your screen, then press ENTER when comfortable")
    
    # Create a loop to read the latest frame from the camera using VideoCapture#read()
    while video.isOpened():
        ret, frame = video.read()
        
        # Convert the frame received from OpenCV to a MediaPipe Image object.
        mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=frame)
        
        # Send live image data to perform face landmarking.
        # The results are accessible via the `result_callback` provided in
        # the `PoseLandmarkerOptions` object.
        # The pose landmarker must be created with the live stream mode.
        landmarker.detect_async(mp_image, frame_timestamp)
        
        # Increment the timestamp
        frame_timestamp += 1
        
        # Draw the output
        cv2.imshow('window', frame)
        
        # Listen for keyboard input
        keyPressed = cv2.waitKey(5)
        if keyPressed == ord('q'): # 'q': Break out of loop and exit program
            break
            
        elif keyPressed == 13: # 'Enter': Calibrate yaw+pitch+roll
            calibrate_yaw_pitch_roll()
            
        elif keyPressed == ord('0'): # '0': Toggle the the remapping
            global remapping_paused
            remapping_paused = not remapping_paused
            print("remapping paused set to: {}".format(remapping_paused))

# Clean up
video.release()
cv2.destroyAllWindows()

---- Calibration -----
Please look straight ahead at your screen, then press ENTER when comfortable
resting yaw is: -0.014276866983158571
resting pitch is: -0.3192686346271227
resting roll is: 0.08348178980813038
remapping paused set to: False


In [13]:
-34 + (-10 * -1)

-24

In [14]:
-34 - -10

-24

In [15]:
-34 - 10

-44

In [16]:
-34 + (10 * -1)

-44